# Pokemon showdown challenge

In [145]:
import pandas as pd
import urllib.request as urllib, zipfile, os

In [146]:
def import_showdown_usage_stats(url, filename): # file url and name for local storage
    # location for datafile
    datafolder = 'data/'
    #download file from url to local storage
    downloaded = urllib.urlopen(url)
    buf = downloaded.read()

    try:
        os.mkdir(datafolder)
    except FileExistsError:
        pass

    with open(datafolder + filename, 'wb') as f:
        f.write(buf)
    # import dataframe
    df = pd.read_csv(datafolder + filename, sep="|",skiprows=5,names=["rank","pokemon","usage","count",],usecols=[1,2,3,4])
    # drop any NaNs we find
    df.dropna(inplace=True)
    # changing data type of the columns
    for k, v in {"rank":int,"pokemon":str, "usage":str, "count":int}.items():
        df[k] = df[k].astype(v)
    # fixing leading/trailing white spaces
    df["pokemon"] = df["pokemon"].str.strip()
    df["usage"] = df["usage"].str.strip()
    # fixing format of string to not use '%'
    df["usage"] = df["usage"].str.rstrip('%')
    # change datatype of usage_%
    df["usage"] = df["usage"].astype(float) / 100
    # get rid of unnecessary columns
    df = df[['pokemon' ,'count' ,'usage']]
    return df

In [171]:
usage_stats = import_showdown_usage_stats(url = "https://www.smogon.com/stats/2023-01/gen9vgc2023series2-0.txt", filename="gen9s2vgc0.txt")
print(usage_stats.shape)
usage_stats.head()

(393, 3)


,pokemon,count,usage
0,Flutter Mane,795029,0.280452
1,Iron Hands,724440,0.255552
2,Amoonguss,705360,0.248821
3,Iron Bundle,663073,0.233904
4,Gholdengo,635767,0.224272


In [172]:
names = ["pokemon","hp","attack","defence","sp.attack","sp.defence","speed","total"]
stats = pd.read_csv("./data/gen_9.csv",names=names)
stats["pokemon"] = stats["pokemon"].str.strip()
print(stats.shape)
stats.head()


(422, 8)


,pokemon,hp,attack,defence,sp.attack,sp.defence,speed,total
0,Koraidon,100,135,115,85,100,135,670
1,Miraidon,100,85,100,135,115,135,670
2,Slaking,150,160,100,95,65,100,670
3,Palafin (Hero Form),100,160,97,106,87,100,650
4,Garchomp,108,130,95,80,85,102,600


In [167]:
df = pd.merge(usage_stats, stats )
df.shape

(368, 10)